In [1]:
import numpy as np
from bound_rewards_g import compute_bound as compute_bound
from f_func import *
from lava_problem import main as lava_problem
import scipy.optimize as optimize


In [2]:


def g(x,c,func_list):
    # c and func_list should have matching dimensions
    y = 0
    for i in range (len(func_list)):
        y += c[i]*eval(func_list[i])(x)
    return y


In [36]:
# Different probabilities of sensor reporting correct state
p_correct_vals = np.linspace(0.01, 0.99, 20) 

# Different rewards for being in a non-lava/goal state
reward_xs = np.array([0.1]) 

#for reward_x in reward_xs:
reward_x = reward_xs[0]

bounds_f_inverse = []

#Setup for finding tightest bounds
func_list = ['Kullback_Leibler', 'Negative_Log', 'Total_Variation', 'Chi_Squared', 'Jensen_Shannon', 'Squared_Hellinger_Distance','Neyman_Chi_Squared']

c0 = np.array([[0,0,1,0,0,0,0],[0,0,1,0,0,0,0],[0,0,1,0,0,0,0],[0,0,1,0,0,0,0],[0,0,1,0,0,0,0],
[0,0,1,0,0,0,0],[0,0,0,0,0,0,1],[0,0,0,0,0,0,1],[0,0,0,0,0,0,1],[0,0,0,0,0,0,1],
[0,0,0,0,0,0,1],[0,0,0,0,0,0,1],[0,0,0,0,0,0,1],[0,0,0,0,0,0,1],[0,0,0,0,0,0,1],
[0,0,0,0,0,0,1],[0,0,0,0,0,0,1],[0,0,0,0,0,0,1],[0,0,0,0,0,0,1],[0,0,0,0,0,0,1]])

def con(c_vec):
	return np.linalg.norm(c_vec)-1

def con1(c_vec):
    return list(c_vec-1e-5)

Nfeval=1
def callbackF(Xi):
    global Nfeval
    print ('{0:4d}{1: 3.6f}{2: 3.6f}{3: 3.6f}{4: 3.6f}{5: 3.6f}{6: 3.6f}{7: 3.6f}{8: 3.6f}'.format(Nfeval,Xi[0],Xi[1],Xi[2],Xi[3],Xi[4],Xi[5],Xi[6], bounds(Xi)))
    Nfeval += 1

cons = [{'type':'ineq','fun': con1}]


In [37]:
p_correct = p_correct_vals[1]
def bounds(c_vec):
    # Figure stuff
    #fig, ax = plt.subplots()
    #colors = cm.jet(np.linspace(1, 0, len(reward_xs)))

    ################################################################################
    # Lava problem
    nx, nu, ny, T, p0, px_x, py_x, R, R0_expected = lava_problem(['--p_correct', str(p_correct), '--reward_x', str(reward_x)])
    ################################################################################


    ################################################################################
    # Compute f-inverse bound
    # t_start = time.time()
    bound_f_inverse = compute_bound(g,c_vec,func_list, nx, nu, ny, T, p0, px_x, py_x, R, R0_expected) 
    # t_end = time.time()
    bounds_f_inverse.append(bound_f_inverse)
    #print("Bound: ", bound_f_inverse)
    # print("Bound computation time: ", t_end - t_start)
    ################################################################################
    return bound_f_inverse

ind = np.where(p_correct_vals == p_correct)[0][0]

res = optimize.minimize(bounds,c0[ind], constraints = cons, callback=callbackF, options={'disp':True})

   1 0.036927 0.726411 1.000000 0.017443 0.197351 0.052546 0.864137 4.073521
   2 0.334486 0.226188 0.998010 0.303082 0.046719 0.000010 0.034779 4.025181
   3 0.000010 0.000096 1.000886 0.000010 0.000010 0.000010 0.000010 3.967701
   4 0.000010 0.000010 1.002508 0.000014 0.000010 0.000010 0.000010 3.967689
   5 0.000010 0.000010 1.002508 0.000014 0.000010 0.000010 0.000010 3.967689
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.9676891679392714
            Iterations: 5
            Function evaluations: 60
            Gradient evaluations: 5


In [38]:
for p_correct in p_correct_vals:
	def bounds(c_vec):
		# Figure stuff
		#fig, ax = plt.subplots()
		#colors = cm.jet(np.linspace(1, 0, len(reward_xs)))

		################################################################################
		# Lava problem
		nx, nu, ny, T, p0, px_x, py_x, R, R0_expected = lava_problem(['--p_correct', str(p_correct), '--reward_x', str(reward_x)])
		################################################################################


		################################################################################
		# Compute f-inverse bound
		# t_start = time.time()
		bound_f_inverse = compute_bound(g,c_vec,func_list, nx, nu, ny, T, p0, px_x, py_x, R, R0_expected) 
		# t_end = time.time()
		bounds_f_inverse.append(bound_f_inverse)
		#print("Bound: ", bound_f_inverse)
		# print("Bound computation time: ", t_end - t_start)
		################################################################################
		return bound_f_inverse

	ind = np.where(p_correct_vals == p_correct)[0][0]

	res = optimize.minimize(bounds,c0[ind], constraints = cons, callback=callbackF, options={'disp':True})


   6 0.000010 0.000010 1.000000 0.391484 0.000010 0.000010 0.000010 4.076787
   7 0.000010 0.000010 1.017793 0.000823 0.000010 0.000010 0.000010 4.053238
   8 0.000010 0.000010 1.000185 0.000014 0.000010 0.000010 0.000010 4.053162
   9 0.000010 0.000010 1.000185 0.000014 0.000010 0.000010 0.000010 4.053162
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.053161745815445
            Iterations: 4
            Function evaluations: 52
            Gradient evaluations: 4
  10 0.036927 0.726411 1.000000 0.017443 0.197351 0.052546 0.864137 4.073521
  11 0.334486 0.226188 0.998010 0.303082 0.046719 0.000010 0.034779 4.025181
  12 0.000010 0.000096 1.000886 0.000010 0.000010 0.000010 0.000010 3.967701
  13 0.000010 0.000010 1.002508 0.000014 0.000010 0.000010 0.000010 3.967689
  14 0.000010 0.000010 1.002508 0.000014 0.000010 0.000010 0.000010 3.967689
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.96768916

SolverError: Solver 'MOSEK' failed. Try another solver, or solve with verbose=True for more information.

In [ ]:
func_list = ['Kullback_Leibler', 'Negative_Log', 'Total_Variation', 'Chi_Squared', 'Jensen_Shannon', 'Squared_Hellinger_Distance','Neyman_Chi_Squared']
c0 = np.array([0,0,1,0,0,0,0])
cons = [{'type':'ineq','fun': con1}]
res = optimize.minimize(bounds,c0, constraints = cons, callback=callbackF, options={'disp':True})


In [27]:
def f_divergence(g,c_vec,func_list,p,q):
    div = cvx.multiply(q,g(p/q,c_vec,func_list))
    return div
    
'''
Compute f-inverse using Relative Entropy Programming
Solves: sup p in [0,1] with D_f(p,q) <= c
'''   
q = 0.5
c = 0.8
p_ber = cvx.Variable(2)
q_ber = np.array([q,1-q])

b = f_divergence(g,c1,func_list,p_ber,q_ber)

constraints = [c >= cvx.sum(b), 0 <= p_ber[0], p_ber[0] <=1, p_ber[1] == 1.0-p_ber[0]]

prob = cvx.Problem(cvx.Maximize(p_ber[0]), constraints)

# Solve problem
#prob.solve(verbose=False, solver=cvx.MOSEK) # solver=cvx.MOSEK



In [ ]:
c1 = [0,0,0,0,0,0,1]
con1(c1)

In [28]:
prob.is_dcp()

True